<img src="https://upload.wikimedia.org/wikipedia/commons/0/06/LMU_Muenchen_Logo.svg" alt="Image" width="200" style="float: right;">

*Gruppe A - Thema 5*

### **RL-Policy-Training unter Vermeidung von Zuständen mit sehr geringer Datendichte**

- Erzeugung eines Datensatzes für das Abschätzung der Datendichte

- Sampeln eines Gym-Environments 
    - Entfernen vordefinierter Bereiche, die z.B. durch die eine optimale Policy laufen würde

- Modell-based RL

- Datendichte im Reward abbilden, aber nicht übergewichten

**Dozent:** [Dr. Michel Tokic](https://www.tokic.com/)

**Studenten:** Maximilian Schieder, Leon Lantz

---


## **Konzept**

Offline Reinforcement Learning!

1. Sampling im Environment
    - Ramdom oder gezielt (Heuristik, Rewards von Environment ändern)
    - Zustand und entsprechende Aktion jedes Schrittes speichern
2. Bestimmung der Datendichte
3. Entfernung von Zuständen mit geringer Datendichte
4. Verwendung des gefilterten Datensatzes für das Training
5. Model-Based RFL ...

Anstatt seltene Zustände komplett zu entfernen, könnte man sie auch weniger stark gewichten?

---

## 📦 **Imports**

- `tensorflow` as `tf` for building and training machine learning models

- `numpy` as `np` for numerical operations and handling arrays

- `random` for generating random numbers and randomizing data

- `gym` for creating and interacting with reinforcement learning environments (OpenAI)

- `pandas` as `pd` for data manipulation and analysis

- `matplotlib.colors` for color normalization in plots

- `matplotlib.pyplot` as `plt` for creating visualizations and plots


In [ ]:
import tensorflow as tf
import numpy as np
import random 
import gymnasium as gym
import numpy as np
import pandas as pd
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import imageio

Every time the code is run, the random numbers generated will be the same, leading to **reproducible results**

In [ ]:
np.random.seed (0)
random.seed(0)
tf.random.set_seed(0)

## 🌍 **Environment**

### 🚗 **Mountain Car**


<img src="https://gymnasium.farama.org/_images/mountain_car.gif" alt="Mountain Car GIF" width="400" >

Gym Environment -> Mountain Car MDP [**(Documentation)**](https://gymnasium.farama.org/environments/classic_control/mountain_car/)
- **Goal**: Reach the goal state at the top of the right hill
- **Problem**: The car's engine is not strong enough to scale the mountain in a single pass. Therefore, the only way to succeed is to drive back and forth to build up momentum
- **Starting Position**: Car starts stochastically at the bottom of a valley

#### **Observation Space**
- 0: position of the car along the x-axis [-1.2, 0.6]
- 1: velocity of the car [-0.07, 0.07]

#### **Action Space**

Discrete deterministic actions
- 0: Accelerate to the left
- 1: Don’t accelerate
- 2: Accelerate to the right

#### **Reward**

The goal is to reach the flag placed on top of the right hill as quickly as possible, as such the agent is penalised with a reward of -1 for each timestep.

#### **Episode End**
- **Termination**: The position of the car is greater than or equal to 0.5 (the goal position on top of the right hill)
- **Truncation**: The length of the episode is 200.





In [ ]:
# for details see: https://github.com/openai/gym/blob/master/gym/envs/classic_control/mountain_car.py
CAR_POS = "carPos"
CAR_VEL = "carVel"
EPISODE = "episode"
STEP = "step"
ACTION = "action"


#OPTIONS: Set by User !!!
RENDERED = True
HEURISTIC = False


def heuristic_policy(velocity):
    if velocity >= 0.0:
        return 2 # Accelerate right
    else:
        return 0 # Accelerate left


def sample_data(episodes=1, seed=0):

    if(RENDERED): env = gym.make("MountainCar-v0", render_mode="human")
    else: env = gym.make("MountainCar-v0")
    
    env.reset()

    ### Create empty Pandas dataset
    transitions = []

    ### SAMPLE DATA
    for episode in range(episodes):
        obs = env.reset()
        step = 0
        done = False

        while step < 200 and not done:
            step += 1

            if(step == 1):
                action = env.action_space.sample()
            else:
                if (HEURISTIC): action = heuristic_policy(obs[1])
                else: action = env.action_space.sample()

            obs, reward, done, _, _ = env.step(action)

            transitions.append(
                {
                    CAR_POS: obs[0],
                    CAR_VEL: obs[1],
                    EPISODE: episode,
                    STEP: step,
                    ACTION: action,
                }
            )
        print("Steps: ", step)

    return pd.DataFrame(transitions)

df = sample_data(episodes=100, seed=0)

**Anderer Ansatz:**

-> Reward ändern. z.B. wenn Car halbwegs gute Ergebnisse erzielt (weit oben auf Hügel) Reward +1 setzen

<p style="color: red;">Zustände nahe des Gipfels sind <b>selten, aber wichtig</b> um das Auto in den Gipfelbereich zu bringen. Das Vermeiden solcher Zustände könnte dazu führen, dass das Modell nicht lernt, wie man erfolgreich den Gipfel erreicht.</p>

### 🌑 **Lunar Lander**

<img src="https://www.gymlibrary.dev/_images/lunar_lander.gif" alt="Lunar Lander GIF" width="400" >

Gym Environment -> Lunar Lander [**(Documentation)**](https://www.gymlibrary.dev/environments/box2d/lunar_lander/)

- **Ziel**: Das Ziel besteht darin, den Lander sicher auf dem Landepad zu landen.

- **Ausgangsposition**: Der Lander startet in der oberen Mitte des Ansichtsbereichs mit einer zufälligen Anfangskraft, die auf seinen Schwerpunkt angewendet wird.

#### **Beobachtungsraum**
Der Beobachtungsraum ist ein 8-dimensionaler Vektor
- [0] : x-Koordinate des Landers
- [1] : y-Koordinate des Landers
- [2] : lineare Geschwindigkeit in x-Richtung
- [3] : lineare Geschwindigkeit in y-Richtung
- [4] : Winkel
- [5] : Winkelgeschwindigkeit
- [6] : linkes Bein in Kontakt mit dem Boden (Boolean)
- [7] : rechtes Bein in Kontakt mit dem Boden (Boolean)

#### **Aktionsraum**
Vier diskrete Aktionen
- [0] : nichts tun
- [1] : linke Orientierungsturbine feuern
- [2] : Hauptturbine feuern
- [3] : rechte Orientierungsturbine feuern

#### **Belohnung**
- Die Belohnung erhöht/verringert sich, je näher/weiter der Lander dem Landepad ist.
- Die Belohnung erhöht/verringert sich, je langsamer/schneller sich der Lander bewegt.
- Die Belohnung verringert sich, je mehr der Lander geneigt ist (Winkel nicht horizontal).
- Die Belohnung erhöht sich um 10 Punkte für jedes Bein, das den Boden berührt.
- Die Belohnung verringert sich um 0,03 Punkte für jeden Frame, in dem eine Seitenturbine feuert.
- Die Belohnung verringert sich um 0,3 Punkte für jeden Frame, in dem die Hauptturbine feuert.

#### **Episodenbeendigung**
- Der Lander stürzt ab (der Landerkörper kommt mit dem Mond in Kontakt).
- Der Lander gerät außerhalb des Ansichtsbereichs (x-Koordinate ist größer als 1).
- Der Lander ist nicht wach. Laut den Box2D-Dokumenten ist ein Körper, der nicht wach ist, ein Körper, der sich nicht bewegt und mit keinem anderen Körper kollidiert.

---

In [ ]:
# Setze den Environment Name
environment_name = 'LunarLander-v2'

#### 🌎 **Zufällige Action-Auswahl**

In [ ]:
# Definiere die Anzahl der Episoden für die Bewertung
num_episodes = 100

# Funktion zur Bewertung des Modells
def evaluate(env, num_episodes=100):
    returns = []
    for episode in range(num_episodes):
        state, info = env.reset()
        done = False
        total_reward = 0
        while not done:
            action = env.action_space.sample()
            state, reward, done, terminated, info = env.step(action)
            total_reward += reward
        returns.append(total_reward)
        #print(f"Episode {episode + 1}: Total Reward: {total_reward}")
    
    # Durchschnittliche Rückgabe berechnen
    average_return = sum(returns) / num_episodes
    print(f"Durchschnittlicher Reward über {num_episodes} Episoden: {average_return}")
    return returns

# Modell bewerten
env = gym.make(environment_name)
returns = evaluate(env, num_episodes=num_episodes)

In [ ]:
env = gym.make(environment_name, render_mode="rgb_array")
episodes = 10

# Funktion zur Aufnahme einer Episode und Speicherung als GIF
for episode in range(episodes):
    output_path = f"results/random/episode_{episode}.gif"
    state, info = env.reset()
    frames = []
    cum_reward = 0
    done = False
    
    while not done:
        # Füge das aktuelle Bild der Frames-Liste hinzu
        frames.append(env.render())
        
        # Zufällige Auswahl einer Aktion
        action = env.action_space.sample()
        state, reward, done, terminated, info = env.step(action)
        cum_reward += reward
    print('Episode:{} Kummulierter Reward:{}'.format(episode, cum_reward))
    
    with imageio.get_writer(output_path, mode='I', duration=0.03) as writer:
        for frame in frames:
            writer.append_data(frame)

env.close()

<p style="color:lightgreen;">Ergebnis:</p>  Bei der Durchführung von zehn Episoden in der <b>Lunar Lander-Umgebung</b>, in denen der Agent zufällig Aktionen auswählt, waren die Ergebnisse erwartungsgemäß schlecht. Der kumulierte Reward ist in der Regel negativ, da zufällige Aktionen selten zu einer erfolgreichen Landung führen. Diese Ergebnisse zeigen deutlich, wie wichtig durchdachte Strategien und gut trainierte Modelle für den Erfolg in komplexen Umgebungen wie Lunar Lander sind.

#### 🏋🏼 **Trainiere ein Modell mit dem A2C-Algorithmus** <b style="color:red;">(Optional)</b>

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = A2C('MlpPolicy', env, ent_coef=0.1, verbose=1)
model.learn(total_timesteps=80000)
model.save("models/A2C_model")

#### 👨‍🏫 **Bewertung der Modelle**

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

#del model #optional
#model = A2C.load('models/A2C_model', env=env)
model = A2C.load('models/a2c-LunarLander-v2')
# Andere Modelle zur Datengenerierung 

Durchschnitt über 100 Episoden

In [ ]:
# Definiere die Anzahl der Episoden für die Bewertung
num_episodes = 100

# Funktion zur Bewertung des Modells
def evaluate_model(model, env, num_episodes=100):
    returns = []
    for episode in range(num_episodes):
        obs = env.reset()
        done = False
        total_reward = 0
        while not done:
            action, _states = model.predict(obs)
            obs, reward, done, info = env.step(action)
            total_reward += reward
        returns.append(total_reward)
        #print(f"Episode {episode + 1}: Total Reward: {total_reward}")
    
    # Durchschnittliche Rückgabe berechnen
    average_return = sum(returns) / num_episodes
    print(f"Durchschnittlicher Reward über {num_episodes} Episoden: {average_return}")
    return returns

# Modell bewerten
returns = evaluate_model(model, env, num_episodes=num_episodes)

Speichere 10 Episoden als GIF

In [ ]:
env = gym.make(environment_name, render_mode="rgb_array")
env = DummyVecEnv([lambda: env])

episodes = 10

# Funktion zur Aufnahme einer Episode und Speicherung als GIF
for episode in range(episodes):
    output_path = f"results/pretrained_model/episode_{episode}.gif"
    state = env.reset()
    frames = []
    cum_reward = 0
    done = False
    
    while not done:
        # Füge das aktuelle Bild der Frames-Liste hinzu
        frames.append(env.render())
        
        # Vorhersage der Aktion durch das Modell
        action, _states = model.predict(state)
        state, reward, done, info = env.step(action)
        cum_reward += reward
    print('Episode:{} Kummulierter Reward:{}'.format(episode, cum_reward))
    
    with imageio.get_writer(output_path, mode='I', duration=0.03) as writer:
        for frame in frames:
            writer.append_data(frame)

# Schließe das Environment
env.close()

# **Autoren**: Maximilian Schieder, Leon Lantz